Test runs for task 2 - ADR extraction

In [1]:
import pickle
import json
from collections import Counter

import pandas as pd 
import pickle 
import re
import numpy as np 
from collections import Counter, defaultdict, OrderedDict
from nltk import word_tokenize, pos_tag
import editdistance
import csv 
from sklearn.metrics import f1_score
import numpy as np
import scipy.stats 
from nltk.corpus import names

from sklearn.model_selection import train_test_split

from weighted_levenshtein import lev, osa, dam_lev

import langid
from nltk.tokenize.treebank import TreebankWordDetokenizer

import os

In [2]:
#import the test data

path = '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/testDataST23_participants.txt'
test_data = pd.read_csv (path, sep = '\t', header = None)
test_data.columns = ['tweet_id', 'text']
print(test_data)

                 tweet_id                                               text
0      540613005508370432           too much ventolin head feels all funny 🙈
1     1012569192325758976                                       @_JMZF Xanax
2      534869994723876865  one of the most common side effects with metfo...
3      541031560406122496  holy shit my spacer seriously amps up albutero...
4      451831506114465792  rivaroxaban is great, until your patient gets ...
5     1002362072829743109  @aphillester I've only heard it on the radio o...
6      539436756387778560  obese women with pcos on a regimen of liraglut...
7     1042415219673362433  Blood thinner, three different pills for arrhy...
8     1034991251169468422  neurotypicals abusing Adderall has made it dif...
9      362423450930905089  @lillybglambert i think it could be lowering m...
10     366756228036177922  @trisha_hornback humira made me have a bad ras...
11    1043830654998327299                                   codein and peggy

In [3]:
txt = list(test_data['text'])

In [4]:
#preprocessing 

class Normalizer (): 
        
    def __init__(self): 
        pass
        
    #to use this function the files need to be sorted in the same folder as the script under /obj_lex/
    def load_obj(self, name):
        with open('/home/dirksonar/Scripts/Project1_lexnorm/preprocessing_pipeline/obj_lex/' + name + '.pkl', 'rb') as f:
            return pickle.load(f, encoding='latin1')
        
    def load_files(self): 
        self.ext_vocab2 = self.load_obj('vocabulary_spelling_unique')
        self.abbr_dict = self.load_obj ('abbreviations_dict')
        self.celex_freq_dict = self.load_obj ('celex_lwrd_frequencies')
        self.celex_list = list(self.celex_freq_dict.keys())
        self.celex_set = set (self.celex_list)
        self.drug_norm_dict = self.load_obj ('drug_normalize_dict')

    def change_tup_to_list(self, tup): 
        thelist = list(tup)
        return thelist
    
    def change_list_to_tup(self,thelist): 
        tup = tuple(thelist)
        return tup
    
#---------Remove URls, email addresses and personal pronouns ------------------
        
    def replace_urls(self,list_of_msgs): 
        list_of_msgs2 = []
        for msg in list_of_msgs: 
            nw_msg = re.sub(
        r'\b' + r'((\(<{0,1}https|\(<{0,1}http|\[<{0,1}https|\[<{0,1}http|<{0,1}https|<{0,1}http)(:|;| |: )\/\/|www.)[\w\.\/#\?\=\+\;\,\&\%_\n-]+(\.[a-z]{2,4}\]{0,1}\){0,1}|\.html\]{0,1}\){0,1}|\/[\w\.\?\=#\+\;\,\&\%_-]+|[\w\/\.\?\=#\+\;\,\&\%_-]+|[0-9]+#m[0-9]+)+(\n|\b|\s|\/|\]|\)|>)',
        ' ', msg)
            list_of_msgs2.append(nw_msg)
        return list_of_msgs2    

    def replace_email(self,list_of_msgs): 
        list_of_msgs2 = []
        for msg in list_of_msgs: 
            nw_msg = re.sub (r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[. ])", ' ', msg) #remove email
            nw_msg2 = re.sub (r"(@[a-zA-Z0-9]+[. ])", ' ', nw_msg) #remove usernames
#             nw_msg3 = re.sub(r"(@ [a-zA-Z0-9]+[. ])", ' ', nw_msg2) #remove usernames
            list_of_msgs2.append(nw_msg2)
        return list_of_msgs2

    def remove_empty (self,list_of_msgs): 
        empty = []
        check_msgs3 =[]
        for a, i in enumerate (list_of_msgs): 
            if len(i) == 0: 
                print('empty')
            else: 
                check_msgs3.append(i)
        return check_msgs3
    

    def create_names_list (self): 
        male_names = names.words('male.txt')
        female_names = names.words('female.txt')
        male_set = set (male_names)
        female_set = set (female_names)
        names_set = male_set.union(female_set) 
        names_list = [] 
        for word in names_set: 
            if (word != 'ned') & (word != 'Ned'): #ned means no evidence and is an important medical term
                word1 = str.lower (word)
                names_list.append(word1) #add the lowered words
                names_list.append(word) #add the capitalized words
        
        self.names_list = names_list
    
    def remove_propernoun_names(self,msg):
        try: 
            nw_msg = [self.change_tup_to_list(token) for token in msg]
            for a, token in enumerate (nw_msg):
                if (token[0] in self.names_list) and ((token[1] == 'NNP') or (token[1]== 'NNPS')): 
                    new_token = token[0].replace (token[0], "-NAME-")
                    nw_msg[a] = [new_token, token[1]]
#             nw_msg2 = [self.change_list_to_tup(token) for token in nw_msg]
            return nw_msg
        except TypeError: 
            pass
    
    def remove_registered_icon (self, msg): 
        nw_msg = re.sub ('\u00AE', '', msg)
        nw_msg2 = re.sub ('\u00E9', 'e', nw_msg)
        return nw_msg2
    
    #this function has been altered because we do not wnat to remove personal pronouns
    def anonymize (self, posts): 
        posts2 = self.replace_urls (posts)
        posts3 = self.replace_email (posts2)
        posts4 = self.remove_empty(posts3)
        posts5 = [self.remove_registered_icon(p) for p in posts4]
#         posts5 = [p.encode('latin-1', errors = 'ignore').decode() for p in posts4]
        posts6 = [word_tokenize (sent) for sent in posts5]
#         posts6 = [pos_tag(sent) for sent in posts5]
#         self.create_names_list()
#         posts7 = [self.remove_propernoun_names (m) for m in posts6]
#         posts8 = []
#         for post in posts7: 
#             tg = [m[0] for m in post]
#             posts8.append(tg)
        return posts6

#---------Convert to lowercase ----------------------------------------------------
    
    def lowercase (self, post):
        post1 = []
        for word in post: 
            word1 = word.lower()
            post1.append (word1)
        return post1

#---------Remove non_English posts -------------------------------------------------    
    def language_identify_basic (self, posts):
        nw = []
        tally = 0
        list_removed = []
        for post in posts: 
            out = langid.classify (post)
            out2 = list(out)
            if out2[0]=='en': 
                nw.append(post)
            else: 
                tally += 1 
                list_removed.append(tuple ([post, out2[0], out2[1]]))
        return nw, tally, list_removed
    
    def language_identify_thres (self, msgs, lang_list, thres):
        nw = []
        tally = 0
        list_removed = []
        for post in msgs: 
            langid.set_languages(lang_list)
            out = langid.classify (post)
            out2 = list(out)
            if out2[0]=='en': 
                nw.append(post)
            elif out2[1] > thres:
                nw.append(post)
            else: 
                tally += 1 
                list_removed.append(tuple ([post, out2[0], out2[1]]))
        return nw, tally, list_removed   

    
    def remove_non_english(self, posts): 
        d = TreebankWordDetokenizer()
        posts2 = [d.detokenize(m) for m in posts]
        
        posts_temp, tally, list_removed = self.language_identify_basic(posts2)        
        lang = []

        for itm in list_removed: 
            lang.append(itm[1])

        c = Counter(lang)

        lang_list = ['en']

        for itm in c.most_common(10): 
            z = list(itm)
            lang_list.append(z[0])
    
        print("Most common 10 languages in the data are:" + str(lang_list))
        posts3, tally_nw, list_removed_nw = self.language_identify_thres(posts2, lang_list, thres = -100)
        return posts3
    
#---------Lexical normalization pipeline (Sarker, 2017) -------------------------------

    def loadItems(self):
        '''
        This is the primary load function.. calls other loader functions as required..
        '''    
        global english_to_american
        global noslang_dict
        global IGNORE_LIST_TRAIN
        global IGNORE_LIST

        english_to_american = {}
        lexnorm_oovs = []
        IGNORE_LIST_TRAIN = []
        IGNORE_LIST = []

        english_to_american = self.loadEnglishToAmericanDict()
        noslang_dict = self.loadDictionaryData()
        for key, value in noslang_dict.items (): 
            value2 = value.lower ()
            value3 = word_tokenize (value2)
            noslang_dict[key] = value3

        return None


    def loadEnglishToAmericanDict(self):
        etoa = {}

        english = open('/home/dirksonar/Scripts/Project1_lexnorm/preprocessing_pipeline/obj_lex/englishspellings.txt')
        american = open('/home/dirksonar/Scripts/Project1_lexnorm/preprocessing_pipeline/obj_lex/americanspellings.txt')
        for line in english:
            etoa[line.strip()] = american.readline().strip()
        return etoa

    def loadDictionaryData(self):
        '''
        this function loads the various dictionaries which can be used for mapping from oov to iv
        '''
        n_dict = {}
        infile = open('/home/dirksonar/Scripts/Project1_lexnorm/preprocessing_pipeline/obj_lex/noslang_mod.txt')
        for line in infile:
            items = line.split(' - ')
            if len(items[0]) > 0 and len(items) > 1:
                n_dict[items[0].strip()] = items[1].strip()
        return n_dict


    #this has been changed becuase we are dealing with twitter data
    def preprocessText(self, tokens, IGNORE_LIST, ignore_username=False, ignore_hashtag=True, ignore_repeated_chars=True, eng_to_am=True, ignore_urls=False):
        '''
        Note the reason it ignores hashtags, @ etc. is because there is a preprocessing technique that is 
            designed to remove them 
        '''
        normalized_tokens =[]
        #print tokens
        text_string = ''
        # NOTE: if nesting if/else statements, be careful about execution sequence...
        for t in tokens:
            t_lower = t.strip().lower()
            # if the token is not in the IGNORE_LIST, do various transformations (e.g., ignore usernames and hashtags, english to american conversion
            # and others..
            if t_lower not in IGNORE_LIST:
                # ignore usernames '@'
                if re.match('@', t) and ignore_username:
                    IGNORE_LIST.append(t_lower)
                    text_string += t_lower + ' '
                #ignore hashtags
                elif re.match('#', t_lower) and ignore_hashtag:
                    IGNORE_LIST.append(t_lower)
                    text_string += t_lower + ' '
                #convert english spelling to american spelling
                elif t.strip().lower() in english_to_american.keys() and eng_to_am:    
                    text_string += english_to_american[t.strip().lower()] + ' '
                #URLS
                elif re.search('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', t_lower) and ignore_urls:
                    IGNORE_LIST.append(t_lower)
                    text_string += t_lower + ' '                
                elif not ignore_repeated_chars and not re.search(r'[^a-zA-Z]', t_lower):
                    # if t_lower only contains alphabetic characters
                    t_lower = re.sub(r'([a-z])\1+', r'\1\1', t_lower)
                    text_string += t_lower + ' '  
                    # print t_lower

                # if none of the conditions match, just add the token without any changes..
                else:
                    text_string += t_lower + ' '
            else:  # i.e., if the token is in the ignorelist..
                text_string += t_lower + ' '
            normalized_tokens = text_string.split()
        # print normalized_tokens
        return normalized_tokens, IGNORE_LIST


    def dictionaryBasedNormalization(self, tokens, I_LIST, M_LIST):
        tokens2 =[]
        for t in (tokens):
            t_lower = t.strip().lower()
            if t_lower in noslang_dict.keys() and len(t_lower)>2:
                nt = noslang_dict[t_lower]
                [tokens2.append(m) for m in nt]

                if not t_lower in M_LIST:
                    M_LIST.append(t_lower)
                if not nt in M_LIST:
                    M_LIST.append(nt)
            else: 
                tokens2.append (t)
        return tokens2, I_LIST, M_LIST
    
#----Using the Sarker normalization functions ----------------------------
#Step 1 is the English normalization and step 2 is the abbreviation normalization

    def normalize_step1(self, tokens, oovoutfile=None):
        global IGNORE_LIST
        global il
        MOD_LIST = []
        # Step 1: preprocess the text
        normalized_tokens, il = self.preprocessText(tokens, IGNORE_LIST)
        normalized_minus_ignorelist = [t for t in normalized_tokens if t not in IGNORE_LIST]
        return normalized_minus_ignorelist
    
    def normalize_step2(self, normalized_tokens, oovoutfile=None): 
        global IGNORE_LIST
        global il
        MOD_LIST = []    
        ml = MOD_LIST
        normalized_tokens, il, ml = self.dictionaryBasedNormalization(normalized_tokens, il, ml)
        return normalized_tokens

    def sarker_normalize (self,list_of_msgs): 
        self.loadItems()
        msgs_normalized = [self.normalize_step1(m) for m in list_of_msgs]
        msgs_normalized2 = [self.normalize_step2(m) for m in msgs_normalized]    
        return msgs_normalized2

#-------Domain specific abreviation expansion ----------------------------
# The list of abbreviations is input as a dictionary with tokenized output  

    def domain_specific_abbr (self, tokens, abbr): 
        post2 = [] 
        for t in tokens:
            if t in abbr.keys(): 
                nt = abbr[t]
                [post2.append(m) for m in nt]
            else: 
                post2.append(t)
        return post2

    def expand_abbr (self, data, abbr): 
        data2 = []
        for post in data: 
            post2 = self.domain_specific_abbr (tokens = post, abbr= abbr)
            data2.append(post2)
        return data2
    
#-------Spelling correction -------------------------------------------------    
    
    def load_files2 (self): 
        #load the edit matrices
        #transpositions
        self.edits_trans = self.load_obj ('weighted_edits_transpositions')
        #deletions 
        self.edits_del = self.load_obj('weighted_edits_deletions')
        #insertions 
        self.edits_ins = self.load_obj('weighted_edits_insertions')
        #substitutions
        self.edits_sub = self.load_obj('weighted_edits_substitutions')
                
        #load the generic dictionary - CHANGE PATH!  
        self.celex_freq_dict = self.load_obj ('celex_lwrd_frequencies')
    
    
    def initialize_weighted_matrices(self): 
    #initialize the cost matrixes for deletions and insertions
        insert_costs = np.ones(128, dtype=np.float64)  # make an array of all 1's of size 128, the number of ASCII characters
        delete_costs = np.ones (128, dtype=np.float64)

        for index,row in self.edits_ins.iterrows(): 
            insert_costs[ord(index)] = row['transformed_frequency']

        for index,row in self.edits_del.iterrows(): 
            delete_costs[ord(index)] = row['transformed_frequency']

        #substitution

        substitute_costs = np.ones((128, 128), dtype=np.float64)
        lst = []
        for index,row in self.edits_sub.iterrows(): 
            z = tuple([row['edit_from'], row['edit_to'], row['transformed_frequency']])
            lst.append (z)
        for itm in lst: 
            itm2 = list(itm)
            try: 
                substitute_costs[ord(itm2[0]), ord(itm2[1])] = itm2[2]
            except IndexError: 
                pass

        #transposition

        transpose_costs = np.ones((128, 128), dtype=np.float64)

        lst = []

        for index,row in self.edits_trans.iterrows(): 
            z = tuple([row['first_letter'], row['second_letter'], row['transformed_frequency']])
            lst.append (z)

        for itm in lst: 
            itm2 = list(itm)
            try: 
                transpose_costs[ord(itm2[0]), ord(itm2[1])] = itm2[2]
            except IndexError: 
                print(itm2)

        return insert_costs, delete_costs, substitute_costs, transpose_costs

    
    def weighted_ed_rel (self, cand, token, del_costs, ins_costs, sub_costs, trans_costs): 
        try: 
            w_editdist = dam_lev(token, cand, delete_costs = del_costs, insert_costs = ins_costs, substitute_costs = sub_costs, transpose_costs = trans_costs)
            rel_w_editdist = w_editdist/len(token)
            return rel_w_editdist
        except UnicodeEncodeError: 
#             print(token)
            IGNORE_LIST.append(token)
            rel_w_editdist = 100
            return rel_w_editdist
    

    def run_low (self, word, voc, func, del_costs, ins_costs, sub_costs, trans_costs): 
        replacement = [' ',100]
        for token in voc: 
            sim = func(word, token, del_costs, ins_costs, sub_costs, trans_costs)
            if sim < replacement[1]:
                replacement[1] = sim
                replacement[0] = token

        return replacement   
    
    
    def spelling_correction (self, post, token_freq_dict, token_freq_ordered, min_rel_freq = 2, max_rel_edit_dist = 0.08): 
        post2 = []
        cnt = 0 

        for a, token in enumerate (post): 
            if self.TRUE_WORD.fullmatch(token):
                if token in self.spelling_corrections:
                    correct = self.spelling_corrections[token] 
                    post2.append(correct)
                    cnt +=1
                    self.replaced.append(token)
                    self.replaced_with.append(correct)

                elif token in self.celex_freq_dict:
                    post2.append(token)

                else:

                    # make the subset of possible candidates
                    freq_word = token_freq_dict[token]
                    limit = freq_word * min_rel_freq
                    subset = [t[0] for t in token_freq_ordered if t[1]>= limit]

                    #compare these candidates with the word        
                    candidate = self.run_low (token, subset, self.weighted_ed_rel, self.delete_costs_nw, self.insert_costs_nw, 
                                         self.substitute_costs_nw, self.transpose_costs_nw)

                #if low enough RE - candidate is deemed good
                    if candidate[1] <= max_rel_edit_dist:
                        post2.append(candidate[0]) 
                        cnt +=1
                        self.replaced.append(token)
                        self.replaced_with.append(candidate[0])
                        self.spelling_corrections [token] = candidate[0]
                    else: 
                        post2.append(token)
            else: post2.append(token)
        self.total_cnt.append (cnt)
        return post2
      
    def initialize_files_for_spelling(self): 
        total_cnt = []
        replaced = []
        replaced_with = []
        spelling_corrections= {}
        return total_cnt, replaced, replaced_with, spelling_corrections
    
    def change_tup_to_list (self, tup): 
        thelist = list(tup)
        return thelist

    def create_token_freq (self, data): 
        flat_data = [item for sublist in data for item in sublist]
        self.token_freq = Counter(flat_data)
        
        token_freq_ordered = self.token_freq.most_common ()
        self.token_freq_ordered2 = [self.change_tup_to_list(m) for m in token_freq_ordered]
    
    def correct_spelling_mistakes(self, data): 
#         data= self.load_obj ('/data/dirksonar/Project1_lexnorm/spelling_correction/output/', 'gistdata_lemmatised')
        self.load_files2()
        self.insert_costs_nw, self.delete_costs_nw, self.substitute_costs_nw, self.transpose_costs_nw = self.initialize_weighted_matrices()
        self.total_cnt, self.replaced, self.replaced_with, self.spelling_corrections= self.initialize_files_for_spelling()
        self.TRUE_WORD = re.compile('[-a-z]+')  # Only letters and dashes  
#         data2 = [word_tokenize(m) for m in data]
        self.create_token_freq(data)
        out = [self.spelling_correction (m, self.token_freq, self.token_freq_ordered2) for m in data]
        return out, self.total_cnt, self.replaced, self.replaced_with, self.spelling_corrections
    
#--------Overall normalization function--------------------------------------
    
    def normalize (self, posts): 
        self.load_files ()
        posts1 = self.anonymize(posts)
        posts2 = [self.lowercase (m) for m in posts1]
#         posts3 = self.remove_non_english (posts2)
#         posts3 = [word_tokenize(m) for m in posts2]
        posts4 = [self.sarker_normalize(posts2)]
        posts5 = [self.expand_abbr(posts4[0], self.abbr_dict)]
#         posts6, total_cnt, replaced, replaced_with, spelling_corrections = self.correct_spelling_mistakes(posts5[0])
        return posts5[0]

    def normalize_extra(self, posts): 
        self.load_files()
        self.loadItems()
        posts2, total_cnt, replaced, replaced_with, spelling_corrections_nw = self.correct_spelling_mistakes(posts)
        posts_ignored = []
        for post in posts2: 
            p2 = [t for t in post if t not in IGNORE_LIST]
            posts_ignored.append(p2)
        return posts_ignored, total_cnt, replaced, replaced_with, spelling_corrections_nw

In [5]:
txt_norm1 = Normalizer().normalize(txt)
# txt_norm2, total_cnt, replaced, replaced_with, spelling_corrections_nw = Normalizer().normalize_extra(txt_norm1)
# print(spelling_corrections_nw)

In [6]:
def remove_punc (post): 
    temp = []
    for word in post: 
        if re.fullmatch (r'[^\w\s]', word) == None: 
            temp.append (word)
        else: 
            pass
    return temp 

# (u"\2026", '')

def post_filter_char (msg):
    final1 = msg.replace('Â', '')
    final2= final1.replace('â€™', '')
    final3 = final2.replace('â€œ', '')
    final4 = final3.replace('â€“', '')
    final5 = final4.replace('â€¦', '')
    final6 = final5.replace('â€', '')
    final7 = final6.replace('...', ' ')
    final8 = final7.replace ('`', '')
    final9 = final8.replace ('ðÿ˜', '')
    final10 = final9.replace ('¡', '')
    final11 = final10.replace ('©', '')
    final12 = final11.replace ('👀🙄', '')
    final13 = final12.replace ( '�', '')
    final14 = final12.replace ('💩', '')
    final15 = re.sub(r'(@ ?[a-zA-Z0-9-_]+[\.: ]?)', '', final14)
    return final15

txt_norm3 = [remove_punc(m) for m in txt_norm1]
d = TreebankWordDetokenizer ()
txt_norm4 = [d.detokenize(m) for m in txt_norm3]
txt_norm5 = [post_filter_char(m) for m in txt_norm4]


In [7]:
txt_norm5

['too much ventolin head feels all funny',
 '_jmzf xanax',
 'one of the most common side effects with metformin is gastrointestinal complaints in most cases start low and titrate up slowly',
 'holy shit my spacer seriously amps up albuterol like i am shaking',
 'rivaroxaban is great until your patient gets a wound and bleeds solidly for 14hours from it',
 "i've only heard it on the radio once but it was the album version with nicotine heroin morphine so yay",
 'obese women with pcos on a regimen of liraglutide metformin and lifestyle changes had significant weight loss',
 "blood thinner three different pills for arrhythmia a strong diuretic and lipitor and you're a worthless russian troll",
 'neurotypicals abusing adderall has made it difficult for people who actually have adhd to gain access to it but whatever hope you finish that final essay 2 hours before class boo',
 'i think it could be lowering my blood sugar too much i take onglyza 5 mg in the morns with metformin 1000 mg',
 "tr

In [8]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(txt_norm5, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/normalized_task2')

# Prediction Time

In [150]:
from flair.models import SequenceTagger
from flair.data import Sentence


In [151]:
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [152]:
# load the model you trained
model = SequenceTagger.load_from_file('/data/dirksonar/Project3_sharedtasks_SMM4H/Task2/flair/bert/final-model.pt')

# create example sentence
sentence = Sentence('I have a headache')

# predict tags and print
model.predict(sentence)
# print(sentence.labels)
print(sentence.to_tagged_string())

2019-04-18 10:51:31,012 loading file /data/dirksonar/Project3_sharedtasks_SMM4H/Task2/flair/bert/final-model.pt
I have a headache <B>


In [34]:
print(sentence.to_dict(tag_type='ner'))

{'text': 'I have a headache', 'labels': [], 'entities': [{'text': 'headache', 'start_pos': 9, 'end_pos': 17, 'type': 'B', 'confidence': 0.9998916387557983}]}


In [153]:
def predict (sent): 
    sent2 = Sentence(sent)
    model.predict(sent2)
    x = sent2.to_dict(tag_type='ner')
    return x

# test = txt_norm5[0:10]
# predicted = [predict(i) for i in txt_norm5]

In [154]:
real_txt = list(test_data['text']) 
predicted_real_txt = [predict(i) for i in real_txt]

In [155]:
save_obj(predicted_real_txt, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/predicted_task2_raw_realtxt')

In [32]:
# print(predicted[0])

{'text': 'too much ventolin head feels all funny', 'labels': [], 'entities': [{'text': 'head', 'start_pos': 18, 'end_pos': 22, 'type': 'B', 'confidence': 0.9962353110313416}, {'text': 'feels', 'start_pos': 23, 'end_pos': 28, 'type': 'I', 'confidence': 0.7620646357536316}, {'text': 'all', 'start_pos': 29, 'end_pos': 32, 'type': 'I', 'confidence': 0.5973029732704163}, {'text': 'funny', 'start_pos': 33, 'end_pos': 38, 'type': 'I', 'confidence': 0.8723652958869934}]}


In [36]:
save_obj(predicted, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/predicted_task2_raw')

In [37]:
# print(predicted)

In [38]:
#reformat 
# def classify (data): 
#     classes = []
#     for sent in data: 
#         if ('<B>' and '<I>') not in sent: 
#             classes.append(0)
#         else:
#             classes.append(1)
#     return classes

#look at later!

In [185]:
# load the second model you trained
model = SequenceTagger.load_from_file('/data/dirksonar/Project3_sharedtasks_SMM4H/Task2/flair/bert2/final-model.pt')

# create example sentence
sentence = Sentence('I have a headache')

# predict tags and print
x = model.predict(sentence)
print(x)

print(sentence.to_tagged_string())

2019-04-18 11:54:09,689 loading file /data/dirksonar/Project3_sharedtasks_SMM4H/Task2/flair/bert2/final-model.pt
[Sentence: "I have a headache" - 4 Tokens]
I have a headache <B>


In [40]:
predicted2 = [predict(i) for i in txt_norm5]

In [41]:
save_obj(predicted2, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/predicted_task2_raw_run2')

In [186]:
predicted2_real_txt = [predict(i) for i in real_txt]

In [187]:
save_obj(predicted2_real_txt, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/predicted_task2_raw_run2_realtxt')

# Reformat 

Checking 

In [43]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f, encoding='latin1')

classes_prediction1 = load_obj ('/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/predicted_class_task2')

In [58]:
classes_prediction_bert1 = []

for i in predicted: 
    if i['entities'] == []: 
        classes_prediction_bert1.append(int(0))
    else: 
        classes_prediction_bert1.append(int(1))

In [59]:
classes_prediction_bert2 = []

for i in predicted2: 
    if i['entities'] == []: 
        classes_prediction_bert2.append(int(0))
    else: 
        classes_prediction_bert2.append(int(1))

In [176]:
classes_prediction_bert1_realtxt = []

for i in predicted_real_txt: 
    if i['entities'] == []: 
        classes_prediction_bert1_realtxt.append(int(0))
    else: 
        classes_prediction_bert1_realtxt.append(int(1))

In [188]:
classes_prediction_bert2_realtxt = []

for i in predicted2_real_txt: 
    if i['entities'] == []: 
        classes_prediction_bert2_realtxt.append(int(0))
    else: 
        classes_prediction_bert2_realtxt.append(int(1))

In [60]:
class_table = pd.concat ([pd.Series (txt_norm5), pd.Series (classes_prediction1).astype('int'), pd.Series(classes_prediction_bert1), pd.Series(classes_prediction_bert2)], axis =1)

In [61]:
class_table.columns =['text', 'task1', 'bert1', 'bert2']

In [62]:
class_table.head()

,text,task1,bert1,bert2
0,too much ventolin head feels all funny,1,1,1
1,_jmzf xanax,0,0,0
2,one of the most common side effects with metfo...,0,1,1
3,holy shit my spacer seriously amps up albutero...,0,1,1
4,rivaroxaban is great until your patient gets a...,1,1,1


In [74]:
x = class_table[class_table['task1'] != class_table['bert1']]

In [75]:
x

,text,task1,bert1,bert2
2,one of the most common side effects with metfo...,0,1,1
3,holy shit my spacer seriously amps up albutero...,0,1,1
6,obese women with pcos on a regimen of liraglut...,0,1,1
8,neurotypicals abusing adderall has made it dif...,0,1,0
14,tuffa_the_track vicodin and oxycontin i hearti...,0,1,1
15,got off the heroin with suboxone then just bui...,0,1,1
16,united health insurance and wouldn't let me bu...,0,1,1
18,but forget my experience change none of your h...,0,1,1
23,why does lunesta ruin everything i eat or drin...,0,1,1
25,i haven't eaten since noon sunday adderall is ...,1,0,0


In [78]:
predicted[28]

{'text': 'my mri scan shows when it happens but it could be areaction"to being on infliximab this time as i did not react on the humira',
 'labels': [],
 'entities': []}

In [82]:
predicted[0]

{'text': 'too much ventolin head feels all funny',
 'labels': [],
 'entities': [{'text': 'head',
   'start_pos': 18,
   'end_pos': 22,
   'type': 'B',
   'confidence': 0.9962353110313416},
  {'text': 'feels',
   'start_pos': 23,
   'end_pos': 28,
   'type': 'I',
   'confidence': 0.7620646357536316},
  {'text': 'all',
   'start_pos': 29,
   'end_pos': 32,
   'type': 'I',
   'confidence': 0.5973029732704163},
  {'text': 'funny',
   'start_pos': 33,
   'end_pos': 38,
   'type': 'I',
   'confidence': 0.8723652958869934}]}

Reformat 

In [80]:
combined_class_prediction = []

for num, i in enumerate(classes_prediction1): 
    if i == int(0): 
        combined_class_prediction.append(int(0))
    elif classes_prediction_bert1[num] == int(0): 
        combined_class_prediction.append(int(0))
    else: 
        combined_class_prediction.append(int(1))

In [180]:
combined_class_prediction_realtxt = []

for num, i in enumerate(classes_prediction1): 
    if i == int(0): 
        combined_class_prediction_realtxt.append(int(0))
    elif classes_prediction_bert1_realtxt[num] == int(0): 
        combined_class_prediction_realtxt.append(int(0))
    else: 
        combined_class_prediction_realtxt.append(int(1))

In [81]:
print (class_table.head())
print(combined_class_prediction[0:6])

                                                text  task1  bert1  bert2
0             too much ventolin head feels all funny      1      1      1
1                                        _jmzf xanax      0      0      0
2  one of the most common side effects with metfo...      0      1      1
3  holy shit my spacer seriously amps up albutero...      0      1      1
4  rivaroxaban is great until your patient gets a...      1      1      1
[1, 0, 0, 0, 1, 0]


In [106]:
exp = predicted[14]
exp

{'text': 'tuffa_the_track vicodin and oxycontin i heartily agree with you daniil as theyve become the biggest drug problem in the usa they are more euphoric than heroin because less desire to puke and are being pushed like dealers push smack opiates should only be prescribed for short term use',
 'labels': [],
 'entities': [{'text': 'euphoric',
   'start_pos': 138,
   'end_pos': 146,
   'type': 'B',
   'confidence': 0.9969794750213623},
  {'text': 'puke',
   'start_pos': 182,
   'end_pos': 186,
   'type': 'B',
   'confidence': 0.9658334255218506}]}

In [107]:
print(exp['entities'][0])

{'text': 'euphoric', 'start_pos': 138, 'end_pos': 146, 'type': 'B', 'confidence': 0.9969794750213623}


In [114]:
# combo_text =[]
# start_end = []
# count = 0
# for num, i in enumerate (exp['entities']):
#     temp_lst = []
#     if num ==0: 
#         start_end[0].append(i['start_pos'])
#         combo_text[0].append(i['text'])
#         start_end[0].append(i['end_pos'])
#     else: 
#         if i['type'] =='B': 
#             count = count+1 
#             combo_text[count].append(i['text'])
#             start_end[count].append(i['start_pos'])
#         elif i['type'] == 'I': 
#             combo_text[count].append(i['text'])
#             start_end.append(i['end_pos'])
# for i in start_end: 
#     nw_i = []
#     nw_i.append(i[0])
#     nw_i.append(i[-1])

def combined_dictionary(data): 
    output = []
    for sent in data: 
        combo_text =[]
        temp_lst = defaultdict(list)
        for num, i in enumerate (sent['entities']):
            if num ==0:
                temp_lst['start'].append(i['start_pos'])
                temp_lst['text'].append(i['text'])
                temp_lst['end'].append(i['end_pos'])
            else: 
                if i['type'] =='B':
                    combo_text.append(temp_lst)
                    temp_lst = defaultdict(list)
                    temp_lst['text'].append(i['text'])
                    temp_lst['start'].append(i['start_pos'])
                    temp_lst['end'].append(i['end_pos'])
                elif i['type'] == 'I': 
                    temp_lst['text'].append(i['text'])
                    temp_lst['end'].append(i['end_pos'])
        combo_text.append(temp_lst)
        output.append(combo_text)
    return output
    

In [117]:
# test = predicted[0:10]

output= combined_dictionary(predicted)
# output

In [118]:
output[0:10]

[[defaultdict(list,
              {'start': [18],
               'text': ['head', 'feels', 'all', 'funny'],
               'end': [22, 28, 32, 38]})],
 [defaultdict(list, {})],
 [defaultdict(list,
              {'start': [54],
               'text': ['gastrointestinal', 'complaints'],
               'end': [70, 81]}),
  defaultdict(list, {'text': ['titrate'], 'start': [110], 'end': [117]})],
 [defaultdict(list,
              {'start': [38],
               'text': ['albuterol', 'like', 'i', 'am', 'shaking'],
               'end': [47, 52, 54, 57, 65]})],
 [defaultdict(list, {'start': [57], 'text': ['bleeds'], 'end': [63]})],
 [defaultdict(list, {})],
 [defaultdict(list,
              {'start': [98], 'text': ['weight', 'loss'], 'end': [104, 109]})],
 [defaultdict(list, {})],
 [defaultdict(list, {'start': [14], 'text': ['abusing'], 'end': [21]})],
 [defaultdict(list,
              {'start': [20],
               'text': ['lowering', 'my', 'blood', 'sugar'],
               'end': [28, 31, 3

In [139]:
z = output[0][0]
print(type(z['text'][0]))

print(type(d.detokenize(z['text'])))

<class 'str'>
<class 'str'>


In [119]:
output2 = combined_dictionary(predicted2)
output2[0:10]

[[defaultdict(list,
              {'start': [18],
               'text': ['head', 'feels', 'all', 'funny'],
               'end': [22, 28, 32, 38]})],
 [defaultdict(list, {})],
 [defaultdict(list, {'start': [71], 'text': ['complaints'], 'end': [81]}),
  defaultdict(list, {'text': ['titrate'], 'start': [110], 'end': [117]})],
 [defaultdict(list,
              {'start': [30],
               'text': ['amps',
                'up',
                'albuterol',
                'like',
                'i',
                'am',
                'shaking'],
               'end': [34, 37, 47, 52, 54, 57, 65]})],
 [defaultdict(list, {'start': [47], 'text': ['wound'], 'end': [52]}),
  defaultdict(list, {'text': ['bleeds'], 'start': [57], 'end': [63]})],
 [defaultdict(list, {})],
 [defaultdict(list,
              {'start': [98], 'text': ['weight', 'loss'], 'end': [104, 109]})],
 [defaultdict(list, {'start': [94], 'text': ['worthless'], 'end': [103]}),
  defaultdict(list, {'text': ['troll'], 'start'

In [143]:
def reformat (class_predict, predictions_dict):
#     temp_df = pd.concat([test_data['tweet_id'], pd.Series(class_predict)], axis = 1)
    
    twt_ids = list(test_data['tweet_id'])
    d = TreebankWordDetokenizer ()
    text_lst = []
    start_lst = []
    end_lst = []
    id_lst = []
    class_lst = []
    
    for num, twt in enumerate (output): 
        if class_predict[num] == 0: 
            class_lst.append(class_predict[num])
            id_lst.append(twt_ids[num])
            text_lst.append('-')
            start_lst.append('-')
            end_lst.append('-')
        elif class_predict[num] ==1 :     
            for inst in twt: 
#                 if len(inst['text']) > 1: 
                nw_txt =d.detokenize(inst['text'])
#                 else: 
#                     nw_txt = inst['text']
                text_lst.append(nw_txt)
                start_lst.append(inst['start'][0])
                end_lst.append(inst['end'][-1])

                id_lst.append(twt_ids[num])
                class_lst.append(class_predict[num])
            
    df_out = pd.concat ([pd.Series (id_lst), pd.Series(start_lst), pd.Series(end_lst), pd.Series(class_lst), pd.Series(text_lst)], axis =1)
    df_out.columns = ['Tweet ID', 'Begin', 'End', 'Class', 'text']
    return df_out
    

In [144]:
df_out_bert1 = reformat(classes_prediction_bert1, output)

In [145]:
df_out_bert1.head()

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,38,1,head feels all funny
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,81,1,gastrointestinal complaints
3,534869994723876865,110,117,1,titrate
4,541031560406122496,38,65,1,albuterol like i am shaking


In [156]:
save_obj (df_out_bert1, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/df_out_bert1_normtext')

In [169]:
def reformat2 (class_predict, predictions_dict):
#     temp_df = pd.concat([test_data['tweet_id'], pd.Series(class_predict)], axis = 1)
    
    twt_ids = list(test_data['tweet_id'])
    d = TreebankWordDetokenizer ()
    text_lst = []
    start_lst = []
    end_lst = []
    id_lst = []
    class_lst = []
    
    for num, twt in enumerate (predictions_dict): 
        if class_predict[num] == 0: 
            class_lst.append(class_predict[num])
            id_lst.append(twt_ids[num])
            text_lst.append('-')
            start_lst.append('-')
            end_lst.append('-')
        elif class_predict[num] ==1 :     
            for inst in twt: 
#                 if len(inst['text']) > 1: 
                nw_txt =d.detokenize(inst['text'])
#                 else: 
#                     nw_txt = inst['text']
                text_lst.append(nw_txt)
                start_lst.append(inst['start'][0])
                end_lst.append(inst['end'][-1])

                id_lst.append(twt_ids[num])
                class_lst.append(class_predict[num])
            
    df_out = pd.concat ([pd.Series (id_lst), pd.Series(start_lst), pd.Series(end_lst), pd.Series(class_lst), pd.Series(text_lst)], axis =1)
    df_out.columns = ['Tweet ID', 'Begin', 'End', 'Class', 'text']
    return df_out


# df_out_bert2 = reformat2(classes_prediction_bert2, output2)

In [157]:
save_obj (df_out_bert2, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/df_out_bert2_normtext')

In [147]:
df_out_bert_adrclass = reformat(combined_class_prediction, output)

In [158]:
save_obj (df_out_bert_adrclass, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/df_out_bert_adrclass_normtext')

In [148]:
df_out_bert_adrclass

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,38,1,head feels all funny
1,1012569192325758976,-,-,0,-
2,534869994723876865,-,-,0,-
3,541031560406122496,-,-,0,-
4,451831506114465792,57,63,1,bleeds
5,1002362072829743109,-,-,0,-
6,539436756387778560,-,-,0,-
7,1042415219673362433,-,-,0,-
8,1034991251169468422,-,-,0,-
9,362423450930905089,20,43,1,lowering my blood sugar


In [165]:
predicted_real_txt

[{'text': 'too much ventolin head feels all funny 🙈',
  'labels': [],
  'entities': [{'text': 'head',
    'start_pos': 18,
    'end_pos': 22,
    'type': 'B',
    'confidence': 0.9912233352661133}]},
 {'text': '@_JMZF Xanax', 'labels': [], 'entities': []},
 {'text': 'one of the most common side effects with metformin is gi complaints – in most cases, start low and titrate up slowly',
  'labels': [],
  'entities': [{'text': 'gi',
    'start_pos': 54,
    'end_pos': 56,
    'type': 'B',
    'confidence': 0.9998642206192017},
   {'text': 'complaints',
    'start_pos': 57,
    'end_pos': 67,
    'type': 'I',
    'confidence': 0.9985345602035522},
   {'text': 'start',
    'start_pos': 85,
    'end_pos': 90,
    'type': 'B',
    'confidence': 0.759167492389679},
   {'text': 'low',
    'start_pos': 91,
    'end_pos': 94,
    'type': 'I',
    'confidence': 0.9594743847846985},
   {'text': 'titrate',
    'start_pos': 99,
    'end_pos': 106,
    'type': 'B',
    'confidence': 0.9999041557312012}

In [174]:
output_real_txt1= combined_dictionary(predicted_real_txt)
output_real_txt1[0:10]
# df_out_bert1_realtxt= reformat(classes_prediction_bert1, output_real_txt1)

[[defaultdict(list, {'start': [18], 'text': ['head'], 'end': [22]})],
 [defaultdict(list, {})],
 [defaultdict(list,
              {'start': [54], 'text': ['gi', 'complaints'], 'end': [56, 67]}),
  defaultdict(list,
              {'text': ['start', 'low'], 'start': [85], 'end': [90, 94]}),
  defaultdict(list,
              {'text': ['titrate', 'up'], 'start': [99], 'end': [106, 109]})],
 [defaultdict(list,
              {'start': [38],
               'text': ['albuterol', 'like', 'i', 'am', 'shaking'],
               'end': [47, 52, 54, 57, 65]})],
 [defaultdict(list, {'start': [58], 'text': ['bleeds'], 'end': [64]})],
 [defaultdict(list, {})],
 [defaultdict(list,
              {'start': [100],
               'text': ['weight', 'loss'],
               'end': [106, 111]})],
 [defaultdict(list, {})],
 [defaultdict(list, {})],
 [defaultdict(list,
              {'start': [36],
               'text': ['lowering', 'my', 'blood', 'sugar'],
               'end': [44, 47, 53, 59]})]]

In [178]:
df_out_bert1_realtxt= reformat2(classes_prediction_bert1_realtxt, output_real_txt1)

df_out_bert1_realtxt.head()

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,22,1,head
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,67,1,gi complaints
3,534869994723876865,85,94,1,start low
4,534869994723876865,99,109,1,titrate up


In [182]:
save_obj (df_out_bert1_realtxt, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/df_out_bert1_realtext')

In [183]:
df_out_bert1_adrclass_realtxt= reformat2(combined_class_prediction_realtxt, output_real_txt1)

df_out_bert1_adrclass_realtxt.head()

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,22,1,head
1,1012569192325758976,-,-,0,-
2,534869994723876865,-,-,0,-
3,541031560406122496,-,-,0,-
4,451831506114465792,58,64,1,bleeds


In [184]:
save_obj (df_out_bert1_adrclass_realtxt, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/df_out_bert1_adrclass_realtext')

In [189]:
output_real_txt2= combined_dictionary(predicted2_real_txt)

In [190]:
df_out_bert2_realtxt= reformat2(classes_prediction_bert2_realtxt, output_real_txt2)
df_out_bert2_realtxt.head()

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,22,1,head
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,67,1,gi complaints
3,534869994723876865,85,94,1,start low
4,534869994723876865,99,109,1,titrate up


In [191]:
save_obj (df_out_bert2_realtxt, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/df_out_bert2_realtext')

# matching of norm and real data

In [ ]:

#df_out_bert1
str.find 

In [194]:
real_txt = test_data['text']
print(real_txt[0:5])

0             too much ventolin head feels all funny 🙈
1                                         @_JMZF Xanax
2    one of the most common side effects with metfo...
3    holy shit my spacer seriously amps up albutero...
4    rivaroxaban is great, until your patient gets ...
Name: text, dtype: object


In [196]:
df_out_bert1['text'].iloc[2] = 'gi complaints'
df_out_bert1.head()

/data/dirksonar/envs/flair/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,38,1,head feels all funny
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,81,1,gi complaints
3,534869994723876865,110,117,1,titrate
4,541031560406122496,38,65,1,albuterol like i am shaking


In [199]:
test_data.columns = ['Tweet ID', 'fulltext']
test_data.head()

,Tweet ID,fulltext
0,540613005508370432,too much ventolin head feels all funny 🙈
1,1012569192325758976,@_JMZF Xanax
2,534869994723876865,one of the most common side effects with metfo...
3,541031560406122496,holy shit my spacer seriously amps up albutero...
4,451831506114465792,"rivaroxaban is great, until your patient gets ..."


In [202]:
print(type(test_data['Tweet ID']))
print(type(df_out_bert1['Tweet ID']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [237]:
out_wtext = pd.merge(df_out_bert1, test_data, on ='Tweet ID', how= 'left')

print (out_wtext['fulltext'].iloc[2])

one of the most common side effects with metformin is gi complaints – in most cases, start low and titrate up slowly


In [204]:
entities = df_out_bert1['text']
print(entities[0:5])

0           head feels all funny
1                              -
2                  gi complaints
3                        titrate
4    albuterol like i am shaking
Name: text, dtype: object


In [206]:
fulltxt = out_wtext['fulltext']

In [238]:
orig_start_lst = out_wtext['Begin'] 
orig_end_lst =out_wtext['End']

In [239]:
start_lst = []
end_lst = []
interesting = []

for num, i in enumerate(entities): 
    if i == '-': 
        start_lst.append('-')
        end_lst.append('-')
    else: 
        sent = fulltxt[num]
        sent2 = sent.lower()
        s = sent2.find(i[:-1])
        if s == -1: 
            s = sent2.find(i[:10])
            if s == -1: 
                s = orig_start_lst[num]
                print(s)
#                 print(sent)
#                 interesting.append(num)
        start_lst.append(s)
        e = s + len(i)
        end_lst.append(e)

56
85
101
31
38
41
165
48
44
32
69
29
32
81
134
39
92
73
79
60
37
67
79


In [231]:
print(len(interesting))

23


In [225]:
print(len(entities))

1859


In [232]:
for x in interesting: 
    print (df_out_bert1_realtxt['text'].iloc[x])

-
tendon tears
happiest
addiction
diarrhoea
hungry
gained about 30 lbs
-
-
-
-
yelling
gained 25 pounds
-
hypoglycemia?
allergies.
gain weight.
afraid to sit up.
emotionally
insomnia.
-
asleep
drowsy!


In [233]:
df_out_bert1_realtxt

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,22,1,head
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,67,1,gi complaints
3,534869994723876865,85,94,1,start low
4,534869994723876865,99,109,1,titrate up
5,541031560406122496,38,65,1,albuterol like i am shaking
6,451831506114465792,58,64,1,bleeds
7,1002362072829743109,-,-,0,-
8,539436756387778560,100,111,1,weight loss
9,1042415219673362433,-,-,0,-


In [234]:
out_wtext_real = pd.merge(df_out_bert1_realtxt, test_data, on ='Tweet ID', how= 'left')
out_wtext_real.head()

,Tweet ID,Begin,End,Class,text,fulltext
0,540613005508370432,18,22,1,head,too much ventolin head feels all funny 🙈
1,1012569192325758976,-,-,0,-,@_JMZF Xanax
2,534869994723876865,54,67,1,gi complaints,one of the most common side effects with metfo...
3,534869994723876865,85,94,1,start low,one of the most common side effects with metfo...
4,534869994723876865,99,109,1,titrate up,one of the most common side effects with metfo...


In [235]:
out_wtext_real

,Tweet ID,Begin,End,Class,text,fulltext
0,540613005508370432,18,22,1,head,too much ventolin head feels all funny 🙈
1,1012569192325758976,-,-,0,-,@_JMZF Xanax
2,534869994723876865,54,67,1,gi complaints,one of the most common side effects with metfo...
3,534869994723876865,85,94,1,start low,one of the most common side effects with metfo...
4,534869994723876865,99,109,1,titrate up,one of the most common side effects with metfo...
5,541031560406122496,38,65,1,albuterol like i am shaking,holy shit my spacer seriously amps up albutero...
6,451831506114465792,58,64,1,bleeds,"rivaroxaban is great, until your patient gets ..."
7,1002362072829743109,-,-,0,-,@aphillester I've only heard it on the radio o...
8,539436756387778560,100,111,1,weight loss,obese women with pcos on a regimen of liraglut...
9,1042415219673362433,-,-,0,-,"Blood thinner, three different pills for arrhy..."


In [216]:
end_lst[0:10]

[38, '-', 67, 106, 65, 64, '-', 111, '-', 21]

In [214]:
x = entities[0]
sent = fulltxt[0]
print(x) 
print(sent)

head feels all funny
too much ventolin head feels all funny 🙈


In [215]:
sent.find(x)

18

In [242]:
out_wtext['Begin'] = pd.Series (start_lst)
out_wtext['End'] = pd.Series (end_lst)

nw_out = out_wtext.drop('fulltext', axis =1)
nw_out.head()

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,38,1,head feels all funny
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,67,1,gi complaints
3,534869994723876865,99,106,1,titrate
4,541031560406122496,38,65,1,albuterol like i am shaking


In [243]:
df_out_bert1.head()

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,38,1,head feels all funny
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,81,1,gi complaints
3,534869994723876865,110,117,1,titrate
4,541031560406122496,38,65,1,albuterol like i am shaking


In [244]:
save_obj (nw_out, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/bert1_task2_matching')

In [248]:
df_out_bert2.head()
out_wtext_bert2.head()

,Tweet ID,Begin,End,Class,text,fulltext
0,540613005508370432,18,38,1,head feels all funny,too much ventolin head feels all funny 🙈
1,1012569192325758976,-,-,0,-,@_JMZF Xanax
2,534869994723876865,57,67,1,complaints,one of the most common side effects with metfo...
3,534869994723876865,99,106,1,titrate,one of the most common side effects with metfo...
4,541031560406122496,30,65,1,amps up albuterol like i am shaking,holy shit my spacer seriously amps up albutero...


In [249]:
df_out_bert2_realtxt.head()

,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,22,1,head
1,1012569192325758976,-,-,0,-
2,534869994723876865,54,67,1,gi complaints
3,534869994723876865,85,94,1,start low
4,534869994723876865,99,109,1,titrate up


In [250]:
#do the same for bert2 and bert1_adr
out_wtext_bert2 = pd.merge(df_out_bert2, test_data, on ='Tweet ID', how= 'left')

entities = out_wtext_bert2['text']
fulltxt = out_wtext_bert2['fulltext']
orig_start_lst = out_wtext_bert2['Begin']

start_lst = []
end_lst = []
interesting = []

for num, i in enumerate(entities): 
    if i == '-': 
        start_lst.append('-')
        end_lst.append('-')
    else: 
        sent = fulltxt[num]
        sent2 = sent.lower()
        s = sent2.find(i[:-1])
        if s == -1: 
            s = sent2.find(i[:10])
            if s == -1: 
                s = orig_start_lst[num]
                print(s)
#                 print(sent)
#                 interesting.append(num)
        start_lst.append(s)
        try: e = s + len(i)
        except TypeError: 
            print(i)
        end_lst.append(e)

out_wtext_bert2['Begin'] = pd.Series (start_lst)
out_wtext_bert2['End'] = pd.Series (end_lst)

nw_out_bert2 = out_wtext_bert2.drop('fulltext', axis =1)
nw_out_bert2.head()

118
56
85
31
38
41
21
165
50
48
78
44
32
30
69
47
118
29
81
134
39
92
86
79
37
67
32
79


,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,38,1,head feels all funny
1,1012569192325758976,-,-,0,-
2,534869994723876865,57,67,1,complaints
3,534869994723876865,99,106,1,titrate
4,541031560406122496,30,65,1,amps up albuterol like i am shaking


In [252]:
save_obj (nw_out_bert2, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/bert2_task2_matching')

In [253]:
#do the same for bert2 and bert1_adr
out_wtext_bert_adr = pd.merge(df_out_bert_adrclass, test_data, on ='Tweet ID', how= 'left')

entities = out_wtext_bert_adr['text']
fulltxt = out_wtext_bert_adr['fulltext']
orig_start_lst = out_wtext_bert_adr['Begin']

start_lst = []
end_lst = []
interesting = []

for num, i in enumerate(entities): 
    if i == '-': 
        start_lst.append('-')
        end_lst.append('-')
    else: 
        sent = fulltxt[num]
        sent2 = sent.lower()
        s = sent2.find(i[:-1])
        if s == -1: 
            s = sent2.find(i[:10])
            if s == -1: 
                s = orig_start_lst[num]
                print(s)
#                 print(sent)
#                 interesting.append(num)
        start_lst.append(s)
        try: e = s + len(i)
        except TypeError: 
            print(i)
        end_lst.append(e)

out_wtext_bert_adr['Begin'] = pd.Series (start_lst)
out_wtext_bert_adr['End'] = pd.Series (end_lst)

nw_out_bert_adr = out_wtext_bert_adr.drop('fulltext', axis =1)
nw_out_bert_adr.head()

56
85
101
38
41
32
81
39
92
73
60
37


,Tweet ID,Begin,End,Class,text
0,540613005508370432,18,38,1,head feels all funny
1,1012569192325758976,-,-,0,-
2,534869994723876865,-,-,0,-
3,541031560406122496,-,-,0,-
4,451831506114465792,58,64,1,bleeds


In [254]:
save_obj (nw_out_bert_adr, '/data/dirksonar/Project3_sharedtasks_SMM4H/testdata/bert1_adr_task2_matching')

In [255]:
nw_out_bert_adr.tail()

,Tweet ID,Begin,End,Class,text
1762,448009964565848064,11,18,1,shaking
1763,466730078697951232,92,129,1,feel like the whole world is spinning
1764,1019697724265025536,-,-,0,-
1765,508758052791529472,-,-,0,-
1766,1039700761284960256,-,-,0,-


In [ ]:
#NOTE FOR MYSELF: NEED TO WATCH THE REPLACEMENT OF PUNCTUATION - MAYBE USE SPACES? INSTEAD OF ''

